In [17]:
import findspark
findspark.init()

In [56]:
import numpy as np
#import arffreader as ar
import matplotlib.pyplot as plt
import sys
import pyspark
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql import SparkSession
from scipy.spatial.distance import pdist, squareform

'''
def findDimensions1(X, y, k, l, L, Mcurr):
	N, d = y.shape
	Dis = [] # dimensions picked for the clusters

	Zis = [] # Z for the remaining dimensions
	Rem = [] # remaining dimensions
	Mselidx = [] # id of the medoid indexing the dimensions in Zis and Rem

	for i in range(len(Mcurr)):
		mi = Mcurr[i]
		print(L[i])# Xij is the average distance from the points in L_i to m_i
		a = X.collect()[mi]
		#Xij = X.groupByKey().map(lambda row: np.abs(row[L[i]] - row[mi]).sum(axis = 0) / len(L[i]))
		Xi = np.zeros(len(a))
		
		for j in range(len(L[i])):
			b = L[i][j] 
			c = X.collect()[b]
			x = np.asarray(c)
			y = np.asarray(a)
			Xg = np.abs(x - y).sum(axis = 0)
			#Xi = np.add(Xg, Xi) #print(Xi) 
		Xij = Xg / len(L[i])
		#Xij = X.map(lambda X: np.abs(X.features - a)) #print(X[L[i]])# Xij here is an array, containing the avg dists in each dimension
		#print(Xij.take(1).foreach(println)) #Xij = np.abs(X[L[i]] - X[mi]).sum(axis = 0) / len(L[i])
		print("XIJ IS :",Xij)#print(Xij)
		Yi = Xij.sum() / d # average distance over all dimensions
		Di = [] # relevant dimensions for m_i
		si = np.sqrt(((Xij - Yi)**2).sum() / (d-1)) # standard deviations
		Zij = (Xij - Yi) / si # z-scores of distances
		print(Zij)
		# pick the smallest two:
		o = np.argsort(Zij)
		#o = o[0]
		print(o)
		Di.append(o[0])
		Di.append(o[1])
		Dis.append(Di)

		for j in range(2,d):
			Zis.append(Zij[o[j]])
			Rem.append(o[j])
			Mselidx.append(i)

	if l != 2:
		# we need to pick the remaining dimensions

		o = np.argsort(Zis)
		
		nremaining = k * l - k * 2
		# print "still need to pick %d dimensions." % nremaining

		# we pick the remaining dimensions using a greedy strategy:
		j = 0
		while nremaining > 0:
			midx = Mselidx[o[j]]
			Dis[midx].append(Rem[o[j]])
			j += 1
			nremaining -= 1

	#print "selected:"
	#print Dis

	return Dis
		
'''
def manhattanSegmentalDist(x, y, Ds):
	""" Compute the Manhattan Segmental Distance between x and y considering
		the dimensions on Ds."""
	dist = 0
	for d in Ds:
		dist += np.abs(x[d] - y[d])
	return dist / len(Ds)

def assignPoints(X, y, Mcurr, Dis):

	assigns = np.ones(y.shape[0]) * -1
	#X = X
	for i in range(y.shape[0]):
		minDist = np.inf
		best = -1
		for j in range(len(Mcurr)):
			b = X.collect()[i]
			a = X.collect()[Mcurr[j]]#dist = X.zipWithIndex().filter(lambda (key,index) : index = i).map(lambda (key,index) : key).collect()
			dist = manhattanSegmentalDist(b.features, a.features, Dis[j])
			#dist = dis.first()#dist = dist.astype(np.int32)
			print("dis is:",dist,"itertion",j,i)
			#dist = manhattanSegmentalDist(X[i], X[Mcurr[j]], Dis[j])
			if dist < minDist:
				minDist = dist
				best = Mcurr[j]

		assigns[i] = best
	#print(assigns)
	return assigns


def evaluateClusters(X, assigns, Dis, Mcurr):

	upperSum = 0.0

	for i in range(len(Mcurr)):		
		C = X[np.where(assigns == Mcurr[i])[0]] # points in cluster M_i
		Cm = C.sum(axis = 0) / C.shape[0] # cluster centroid
		Ysum = 0.0

		for d in Dis[i]:
			# avg dist to centroid along dim d:
			Ysum += np.sum(np.abs(C[:,d] - Cm[d])) / C.shape[0]
		wi = Ysum / len(Dis[i])

		upperSum += C.shape[0] * wi

	return upperSum / X.shape[0]

def computeBadMedoids(X, assigns, Dis, Mcurr, minDeviation):
	N, d = X.shape
	k = len(Mcurr)
	Mbad = []
	counts = [len(np.where(assigns == i)[0]) for i in Mcurr]
	cte = int(np.ceil((N / k) * minDeviation))

	# get the medoid with least points:
	Mbad.append(Mcurr[np.argsort(counts)[0]])

	for i in range(len(counts)):
		if counts[i] < cte and Mcurr[i] not in Mbad:
			Mbad.append(Mcurr[i])

	return Mbad

def proclus(X, y, k = 2, l = 3, minDeviation = 0.1, A = 30, B = 3, niters = 30, seed = 1234):
	""" Run PROCLUS on a database to obtain a set of clusters and 
		dimensions associated with each one.
		Parameters:
		----------
		- X: 	   		the data set
		- k: 	   		the desired number of clusters
		- l:	   		average number of dimensions per cluster
		- minDeviation: for selection of bad medoids
		- A: 	   		constant for initial set of medoids
		- B: 	   		a smaller constant than A for the final set of medoids
		- niters:  		maximum number of iterations for the second phase
		- seed:    		seed for the RNG
	"""
	np.random.seed(seed)

	N, d = y.shape

	if B > A:
		raise Exception("B has to be smaller than A.")

	if l < 2:
		raise Exception("l must be >=2.")

	###############################
	# 1.) Initialization phase
	###############################

	# first find a superset of the set of k medoids by random sampling
	idxs = np.arange(N)
	np.random.shuffle(idxs)
	S = idxs[0:(A*k)]
	M = greedy2(X, S, B * k)
	print("medoids are ",M)
	###############################
	# 2.) Iterative phase
	###############################

	BestObjective = np.inf

	# choose a random set of k medoids from M:
	Mcurr = np.random.permutation(M)[0:k] # M current
	Mbest = None # Best set of medoids found

	D = squareform(pdist(y)) # precompute the euclidean distance matrix
	#print(D)
	it = 0 # iteration counter
	L = [] # locality sets of the medoids, i.e., points within delta_i of m_i.
	Dis = [] # important dimensions for each cluster
	assigns = [] # cluster membership assignments

	while True:
		it += 1
		L = []#print(it)

		for i in range(len(Mcurr)):
			mi = Mcurr[i]
			# compute delta_i, the distance to the nearest medoid of m_i:
			di = D[mi,np.setdiff1d(Mcurr, mi)].min()
			#print(di)# compute L_i, points in sphere centered at m_i with radius d_i
			L.append(np.where(D[mi] <= di)[0])

		#print(L)# find dimensions:
		Dis = findDimensions2(X, y, k, l, L, Mcurr)
		print("dimensions are:",Dis)
		# form the clusters:
		assigns = assignPoints(X, y, Mcurr, Dis)
		
		# evaluate the clusters:
		ObjectiveFunction = evaluateClusters(X, assigns, Dis, Mcurr)
		print(ObjectiveFunction)
		badM = [] # bad medoids

		Mold = Mcurr.copy()

		if ObjectiveFunction < BestObjective:
			BestObjective = ObjectiveFunction
			Mbest = Mcurr.copy()
			# compute the bad medoids in Mbest:
			badM = computeBadMedoids(X, assigns, Dis, Mcurr, minDeviation)
			print ("bad medoids:")
			print (badM)

		if len(badM) > 0:
			# replace the bad medoids with random points from M:
			print ("old mcurr:")
			print (Mcurr)
			Mavail = np.setdiff1d(M, Mbest)
			newSel = np.random.choice(Mavail, size = len(badM), replace = False)
			Mcurr = np.setdiff1d(Mbest, badM)
			Mcurr = np.union1d(Mcurr, newSel)
			print ("new mcurr:")
			print (Mcurr)

		print ("finished iter: %d" % it)

		if np.allclose(Mold, Mcurr) or it >= niters:
			break

	print ("finished iterative phase...")

	###############################
	# 3.) Refinement phase
	###############################

	# compute a new L based on assignments:
	L = []
	for i in range(len(Mcurr)):
		mi = Mcurr[i]
		L.append(np.where(assigns == mi)[0])

	Dis = findDimensions(X, y, k, l, L, Mcurr)
	assigns = assignPoints(X, Mcurr, Dis)

	# handle outliers:

	# smallest Manhattan segmental distance of m_i to all (k-1)
	# other medoids with respect to D_i:
	deltais = np.zeros(k)
	for i in range(k):
		minDist = np.inf
		for j in range(k):
			if j != i:
				dist = manhattanSegmentalDist(X[Mcurr[i]], X[Mcurr[j]], Dis[i])
				if dist < minDist:
					minDist = dist
		deltais[i] = minDist

	# mark as outliers the points that are not within delta_i of any m_i:
	for i in range(len(assigns)):
		clustered = False
		for j in range(k):
			d = manhattanSegmentalDist(X[Mcurr[j]], X[i], Dis[j])
			if d <= deltais[j]:
				clustered = True
				break
		if not clustered:
			
			#print "marked an outlier"
			assigns[i] = -1

	return (Mcurr, Dis, assigns)

def computeBasicAccuracy(pred, expect):
	""" Computes the clustering accuracy by assigning
		a class to each cluster based on majority
		voting and then comparing with the expected
		class. """

	if len(pred) != len(expect):
		raise Exception("pred and expect must have the same length.")

	uclu = np.unique(pred)

	acc = 0.0

	for cl in uclu:
		points = np.where(pred == cl)[0]
		pclasses = expect[points]
		uclass = np.unique(pclasses)
		counts = [len(np.where(pclasses == u)[0]) for u in uclass]
		mcl = uclass[np.argmax(counts)]
		acc += np.sum(np.repeat(mcl, len(points)) == expect[points])

	acc /= len(pred)

	return acc


In [57]:
def dist(x,y):   
    return numpy.sqrt(numpy.sum((x-y)**2))

def greedy2(X, S, k):
	# remember that k = B * k here...

	M = [np.random.permutation(S)[0]] # M = {m_1}, a random point in S
	# print M

	A = np.setdiff1d(S, M) # A = S \ M
	#print(len(A))
	dists = np.zeros(len(A))

	#for i in range(len(A)):
	y = X.collect()[M[0]]
	#print(y.features)#dis = X.filter
	dist = X.filter(lambda x: any(x[9] == A[j] for j in range(len(A))))#.map(lambda x: np.linalg.norm(x.features-y.features))
	#print(dist.count())
	dists = dist.map(lambda x: np.linalg.norm(x.features-y.features)).collect()
	print(dists)
	
	#dists[i] = np.linalg.norm(X[A[i]] - X[M[0]]) # euclidean distance

	# print dists

	for i in range(1, k):
		# choose medoid m_i as the farthest from previous medoids

		midx = np.argmax(dists)
		mi = A[midx]
		
		M.append(mi)
				
		# update the distances, so they reflect the dist to the closest medoid:
		for j in range(len(A)):
			y = X.collect()[mi]
			b = X.collect()[A[j]]#filter(lambda x: x[9] == A[j])
			#print(b.features)#.map(lambda x: np.linalg.norm(X.features - y.features)).collect()
			dists[j] = min(dists[j], np.linalg.norm(b.features - y.features))
		
		# remove mi entries from A and dists:
		A = np.delete(A, midx)
		dists = np.delete(dists, midx)

	return np.array(M)


In [58]:
def findDimensions2(X, y, k, l, L, Mcurr):
	N, d = y.shape
	Dis = [] # dimensions picked for the clusters
	#print(len(L[0]))
	Zis = [] # Z for the remaining dimensions
	Rem = [] # remaining dimensions
	Mselidx = [] # id of the medoid indexing the dimensions in Zis and Rem

	for i in range(len(Mcurr)):
		mi = Mcurr[i]
		m = X.collect()[mi]
		Davg = X.filter(lambda x: any(x[9] == L[i][j] for j in range(len(L[i]))))# Xij is the average distance from the points in L_i to m_i
		#print(Davg.count())
		Xij = Davg.map(lambda x: np.abs(x.features - m.features)).reduce(lambda x,y: np.true_divide(np.add(x,y),len(L[i])))#.reduce() # Xij here is an array, containing the avg dists in each dimension
		#print(Xij)
		#Xij = np.abs(X[L[i]] - X[mi]).sum(axis = 0) / len(L[i])
		print("XIJ IS :",Xij)
		Yi = Xij.sum() / d # average distance over all dimensions
		Di = [] # relevant dimensions for m_i
		si = np.sqrt(((Xij - Yi)**2).sum() / (d-1)) # standard deviations
		Zij = (Xij - Yi) / si # z-scores of distances

		o = np.argsort(Zij)# pick the smallest two:
		print(o)
		Di.append(o[0])
		Di.append(o[1])
		Dis.append(Di)

		for j in range(2,d):
			Zis.append(Zij[o[j]])
			Rem.append(o[j])
			Mselidx.append(i)

	if l != 2:
		# we need to pick the remaining dimensions

		o = np.argsort(Zis)
		
		nremaining = k * l - k * 2
		# print "still need to pick %d dimensions." % nremaining

		# we pick the remaining dimensions using a greedy strategy:
		j = 0
		while nremaining > 0:
			midx = Mselidx[o[j]]
			Dis[midx].append(Rem[o[j]])
			j += 1
			nremaining -= 1

	#print "selected:"
	#print Dis

	return Dis
		

In [59]:
import numpy as np
from numpy import genfromtxt
#import adjrand
#import arffreader as ar

##### read data#####
"""
#X = spark.read.text(sys.argv[1]).rdd.map(lambda r: r[0]).mapPartitions(readPointBatch).cache()
#y, sup = ar.readarff("/usr/lib/spark/examples/src/main/python/Major_proj/datasets/Archive/leukemia2.csv") #read from arff
#y = genfromtxt('/usr/lib/spark/examples/src/main/python/Major_proj/datasets/Archive/leukemia1.csv',delimiter=',',skip_header=1) #path to csv file
y = pd.read_csv('/usr/lib/spark/examples/src/main/python/Major_proj/datasets/Archive/leukemia1.csv',delimiter=',')
# change label values here
#print(y)
print(y.dtypes)
#df =  df.withColumn("cls", df["CLASS"].getItem(0).cast("string")) 
y = y.replace({'ALL': 0, 'AML': 1})
sup = [item[-1] for item in y]
sup = np.array(sup)
sup = sup.ravel()
print(y)
"""
g = pd.read_csv('/usr/lib/spark/examples/src/main/python/Major_proj/datasets/Archive/diabetes.csv',delimiter=',')
#print(y)
y = g.iloc[:, :-1]
#dftt = dft.toDF()
#print(type(dftt))
X = dft.rdd
print(X.count())
print("file is of type",X)
Dims = [0,1]
#plotter.plotDataset(X, D = Dims) # plot 0-1 dimensions

R = 1 # toggle run proclus
RS = 0 # toggle use random seed

if R: # run proclus
	rseed = 9028844
	if RS:
		rseed = np.random.randint(low = 0, high = 1239831)

	print ("Using seed %d" % rseed)
	k = 2
	l = 5
	M, D, A = proclus(X, y, k, l, seed = rseed)
	print(A)
	#print "Accuracy: %.4f" % computeBasicAccuracy(A, sup)
	#print "Adjusted rand index: %.4f" % adjrand.computeAdjustedRandIndex(A, sup)

	#plotter.plotClustering(X, M, A, D = Dims)


768
file is of type MapPartitionsRDD[12] at javaToPython at NativeMethodAccessorImpl.java:0
Using seed 9028844
[457.0306528357313, 438.6111741540644, 424.2274126114329, 381.1429979306471, 377.1769444346898, 368.3322316188832, 355.1193037830944, 323.3036621469794, 320.6314855809329, 294.9615217006924, 277.01973617321136, 269.0780862494375, 253.25870239510786, 228.95145453531566, 735.1065271264796, 231.07565919372638, 201.70526440577396, 190.09708446438142, 199.01539691518505, 182.10427853209802, 524.5896156844912, 137.66374484632573, 223.11900615405156, 246.47828843116974, 124.86669147302597, 120.15847878948175, 113.01449921080228, 93.28711701040662, 249.990188370825, 27.422940925515494, 51.55594757907811, 81.00683260691409, 75.57436008447637, 121.98933253887259, 73.66452229605935, 82.94538028719748, 108.17017414612964, 117.05063007942672, 230.2982750098025, 127.77279762922856, 279.29520224527425, 316.45626242280406, 164.08029479327183, 230.88541362609945, 170.1088457808457, 241.2063415

TypeError: Indices must be of type integer, got type <class 'numpy.int64'>

In [4]:
def readPointBatch(iterator):
    strs = list(iterator)
    matrix = np.zeros((len(strs), D + 1))
    for i, s in enumerate(strs):
        matrix[i] = np.fromstring(s.replace(',', ' '), dtype=np.float32, sep=' ')
    return [matrix]


In [5]:
#preprocessing
from numpy import array
from math import sqrt
from numpy import genfromtxt
import pandas as pd
from pyspark.mllib.clustering import KMeans
from pyspark.sql import SQLContext

spark = SparkSession.builder.appName("proclus").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

#Load and parse the data
#dataset = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load('/usr/lib/spark/examples/src/main/python/Major_proj/datasets/Archive/glass.csv')
#df = pd.read_csv('/usr/lib/spark/examples/src/main/python/Major_proj/datasets/Archive/glass.csv',delimiter=',') # path to file here
dataset = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load('/usr/lib/spark/examples/src/main/python/Major_proj/diabetes.csv')
df = pd.read_csv('/usr/lib/spark/examples/src/main/python/Major_proj/diabetes.csv',delimiter=',') # path to file here

a = len(df.columns)
#print(a)
label = df.as_matrix(columns=df.columns[a-1:])
label = label.ravel()
dataset = dataset.na.drop()
#print(label)
dataset.dtypes
'''
from pyspark.sql.types import FloatType
dataset = dataset.withColumn("id", dataset["id"].cast(FloatType()))
dataset = dataset.withColumn("Pregnancies", dataset["Pregnancies"].cast(FloatType()))
dataset = dataset.withColumn("Glucose", dataset["Glucose"].cast(FloatType()))
dataset = dataset.withColumn("BloodPressure", dataset["BloodPressure"].cast(FloatType()))
dataset = dataset.withColumn("SkinThickness", dataset["SkinThickness"].cast(FloatType()))
dataset = dataset.withColumn("Insulin", dataset["Insulin"].cast(FloatType()))
dataset = dataset.withColumn("DiabetesPedigreeFunction", dataset["DiabetesPedigreeFunction"].cast(FloatType()))
dataset = dataset.withColumn("BMI", dataset["BMI"].cast(FloatType()))
dataset = dataset.withColumn("Age", dataset["Age"].cast(FloatType()))
'''
dataset.dtypes

/opt/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


[('id', 'string'),
 ('Pregnancies', 'string'),
 ('Glucose', 'string'),
 ('BloodPressure', 'string'),
 ('SkinThickness', 'string'),
 ('Insulin', 'string'),
 ('BMI', 'string'),
 ('DiabetesPedigreeFunction', 'string'),
 ('Age', 'string'),
 ('Outcome', 'string')]

In [6]:
#preprocessing
dft = dataset.select(*(dataset[c].cast("float").alias(c) for c in dataset.columns[1:]))

from pyspark.sql.functions import monotonically_increasing_id 

dft = dft.select("*").withColumn("id", monotonically_increasing_id())

#split outcome
colm = ("'class'")
dft = dft.drop(colm)
#print(dft.head())
from pyspark.ml.feature import VectorAssembler # in spark api we have to convert data to vectors in order to run the model.

# creating an instance of the vector assembler 
assembler = VectorAssembler(inputCols = dft.columns, outputCol = 'features')

# transforming dataframe into vector assembler 
final_df = assembler.transform(dft)
#final_df.drop('Outcome')
#final_df = final_df.rdd.map(tuple)
#df = final_df.ix[:,-1]
#df = final_df.select("features").rdd.map(lambda x: x).collect()
#print(df.show())
#df = final_df.select('features')
#df = dfth[0].features.toArray()     # to array
#df = df.rdd
#dft = final_df.select("features") 
dft = final_df
dft

DataFrame[Pregnancies: float, Glucose: float, BloodPressure: float, SkinThickness: float, Insulin: float, BMI: float, DiabetesPedigreeFunction: float, Age: float, Outcome: float, id: bigint, features: vector]

In [7]:
def greedy(X, S, k):
	# remember that k = B * k here...

	M = [np.random.permutation(S)[0]] # M = {m_1}, a random point in S
	# print M

	A = np.setdiff1d(S, M) # A = S \ M
	dists = np.zeros(len(A))
	a = M[0]
	print(X)#values = X.first().collect()
	medi = X.collect()[a]
	#print(medi)
	for i in range(len(A)):
		b = A[i]
		itr = X.collect()[b]  
		#print(itr)
		x = np.asarray(medi)
		y = np.asarray(itr)
		dists[i] = np.linalg.norm(x[0] - y[0]) # euclidean distance

	print (dists)

	for i in range(1, k):
		# choose medoid m_i as the farthest from previous medoids

		midx = np.argmax(dists)
		mi = A[midx]
		
		M.append(mi)
		mid = X.collect()[mi]
		# update the distances, so they reflect the dist to the closest medoid:
		for j in range(len(A)):
			b = A[j]
			itr_dist = X.collect()[b] 
			x = np.asarray(mid)
			y = np.asarray(itr_dist)
			dists[j] = min(dists[j], np.linalg.norm(x[0] - y[0]))
		
		# remove mi entries from A and dists:
		A = np.delete(A, midx)
		dists = np.delete(dists, midx)
	print(np.array(M))
	return np.array(M)


In [8]:
def findDimensions(X, y, k, l, L, Mcurr):
	N, d = y.shape
	Dis = [] # dimensions picked for the clusters

	Zis = [] # Z for the remaining dimensions
	Rem = [] # remaining dimensions
	Mselidx = [] # id of the medoid indexing the dimensions in Zis and Rem

	for i in range(len(Mcurr)):
		mi = Mcurr[i]
		#a = X.collect()[mi]
		# Xij is the average distance from the points in L_i to m_i
		#X1 = X.zipWithIndex()
		#print(X1.first())
		Xij = X.filter(lambda X : X[0]).map(absolute(X,L,mi)).collect()  # dimension
		#Xij = X.rdd.zipWithIndex().filter(lambda (key,index) : index == L[i]).map(lambda (key,index): np.abs(X[L[i]] - X[mi])/len(L[i])).collect()  # dimension
		#Xij = np.abs(X[L[i]] - X[mi]).sum(axis = 0) / len(L[i])
		print("XIJ IS :",Xij)
		Yi = Xij.sum() / d # average distance over all dimensions
		Di = [] # relevant dimensions for m_i
		si = np.sqrt(((Xij - Yi)**2).sum() / (d-1)) # standard deviations
		Zij = (Xij - Yi) / si # z-scores of distances

		o = np.argsort(Zij)# pick the smallest two:
		print(o)
		Di.append(o[0])
		Di.append(o[1])
		Dis.append(Di)

		for j in range(2,d):
			Zis.append(Zij[o[j]])
			Rem.append(o[j])
			Mselidx.append(i)

	if l != 2:
		# we need to pick the remaining dimensions

		o = np.argsort(Zis)
		
		nremaining = k * l - k * 2
		# print "still need to pick %d dimensions." % nremaining

		# we pick the remaining dimensions using a greedy strategy:
		j = 0
		while nremaining > 0:
			midx = Mselidx[o[j]]
			Dis[midx].append(Rem[o[j]])
			j += 1
			nremaining -= 1

	#print "selected:"
	#print Dis

	return Dis
		

def absolute(X,L,mi):
    g = []
    for j in range(len(L)):
        print(X.collect()[j])
        g = np.abs(np.asarray(X.collect()[j]) - np.asarray(X.collect()[mi])).sum(axis = 0)/len(L)
        print(g)
        y.append(g)
    return y


In [26]:
dat = dataset.rdd
dat.first()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4, localhost, executor driver): java.io.IOException: Cannot run program "usr/bin/python3.5": error=2, No such file or directory
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:197)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:122)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:95)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:108)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoin

In [ ]:
y = dat.filter(lambda x: x[0] == 54 or x[0] == 4)
y.first()